In [1]:
import json
import math

In [2]:
qasm = '''

OPENQASM 2.0;
include "hqslib1.inc";
qreg q0[6];
qreg q1[6];
qreg q2[6];


qreg a0[2];


creg init0[2];
creg meas0[6];

creg magic0[6];
creg magic1[6];



creg hcheck0[2];
creg hcheck1[2];

creg ycheck0[1];
creg ycheck1[1];


creg xcheck0[2];
creg xcheck1[2];


creg flags0[2];
creg flags1[2];

creg twirl[2];


creg scratch[6];
creg rng_index[32];


gate chh() a,b
{
   ry(-pi/4) b;
   cz a,b;
   ry(pi/4) b;
}


// Generating a random binary string of length 2 on H1-1 for twirling
RNGseed(1);
rng_index = JOB_shotnum;
RNGindex(rng_index);
RNGbound(4);
twirl = RNGnum();


h q0[0];
cx q0[0], a0[0];
cx q0[0], q0[1];
cx q0[0], q0[4];
cx q0[0], q0[5];
cx q0[0], a0[0];
measure a0[0] -> init0[0];
reset a0[0];

h q0[2];
cx q0[2], a0[1];
cx q0[2], q0[3];
cx q0[2], q0[4];
cx q0[2], q0[5];
cx q0[2], a0[1];
measure a0[1] -> init0[1];
reset a0[1];



// Begin First Magic State injection
ry(pi/4) q1[0];

h q1[1];
sdg q1[1];

h q1[2];
h q1[4];
cx q1[2], q1[3];
cx q1[4], q1[5];
cx q1[2], q1[0];
cx q1[3], q1[1];
cx q1[0], q1[4];
cx q1[1], q1[5];
cx q1[4], q1[2];
cx q1[5], q1[3];


h a0[0];
cx a0[0], a0[1];


chh a0[0], q1[0];
cx a0[1], q1[1];
chh a0[1], q1[1];
chh a0[0], q1[2];
cx a0[1], q1[3];
chh a0[1], q1[3];
chh a0[0], q1[4];
cx a0[1], q1[5];
chh a0[1], q1[5];



cx a0[0], a0[1];
t a0[0];
h a0[0];

measure a0 -> hcheck0;
reset a0;


h a0[0];
cy a0[0], q1[1];
cy a0[0], q1[3];
cy a0[0], q1[5];
h a0[0];
measure a0[0] -> ycheck0[0];
reset a0[0];




// Twirl here
if(twirl[0] == 1)h q1;
if(twirl[0] == 1)x q1[1];
if(twirl[0] == 1)x q1[3];
if(twirl[0] == 1)x q1[5];

cy q1, q0;

sdg q1;
h q1;
measure q1 -> magic0;
reset q1;


scratch[0] = magic0[0]^magic0[2];
scratch[0] = scratch[0]^magic0[4];

if(scratch[0] == 0) ry(-pi/2) q0;

// We must extract the X syndromes after rotation gadget
h a0[0];
cx a0[0], q0[0];
cx a0[0], a0[1];
cx a0[0], q0[1];
cx a0[0], q0[2];
cx a0[0], a0[1];
cx a0[0], q0[3];
h a0[0];
measure a0[0] -> xcheck0[0];
measure a0[1] -> flags0[0];
reset a0;


h a0[0];
cx a0[0], q0[2];
cx a0[0], a0[1];
cx a0[0], q0[3];
cx a0[0], q0[4];
cx a0[0], a0[1];
cx a0[0], q0[5];
h a0[0];
measure a0[0] -> xcheck0[1];
measure a0[1] -> flags0[1];
reset a0;





// Begin Second Magic State injection
ry(pi/4) q2[0];

h q2[1];
sdg q2[1];

h q2[2];
h q2[4];
cx q2[2], q2[3];
cx q2[4], q2[5];
cx q2[2], q2[0];
cx q2[3], q2[1];
cx q2[0], q2[4];
cx q2[1], q2[5];
cx q2[4], q2[2];
cx q2[5], q2[3];


h a0[0];
cx a0[0], a0[1];


chh a0[0], q2[0];
cx a0[1], q2[1];
chh a0[1], q2[1];
chh a0[0], q2[2];
cx a0[1], q2[3];
chh a0[1], q2[3];
chh a0[0], q2[4];
cx a0[1], q2[5];
chh a0[1], q2[5];



cx a0[0], a0[1];
t a0[0];
h a0[0];

measure a0 -> hcheck1;
reset a0;


h a0[0];
cy a0[0], q2[1];
cy a0[0], q2[3];
cy a0[0], q2[5];
h a0[0];
measure a0[0] -> ycheck1[0];
reset a0[0];




// Twirl here
if(twirl[1] == 1)h q2;
if(twirl[1] == 1)x q2[1];
if(twirl[1] == 1)x q2[3];
if(twirl[1] == 1)x q2[5];

cy q2, q0;

sdg q2;
h q2;
measure q2 -> magic1;
reset q2;


scratch[0] = magic1[0]^magic1[2];
scratch[0] = scratch[0]^magic1[4];

if(scratch[0] == 0) ry(-pi/2) q0;

//We must extract the X syndromes after rotation gadget
h a0[0];
cx a0[0], q0[0];
cx a0[0], a0[1];
cx a0[0], q0[1];
cx a0[0], q0[2];
cx a0[0], a0[1];
cx a0[0], q0[3];
h a0[0];
measure a0[0] -> xcheck1[0];
measure a0[1] -> flags1[0];
reset a0;


h a0[0];
cx a0[0], q0[2];
cx a0[0], a0[1];
cx a0[0], q0[3];
cx a0[0], q0[4];
cx a0[0], a0[1];
cx a0[0], q0[5];
h a0[0];
measure a0[0] -> xcheck1[1];
measure a0[1] -> flags1[1];
reset a0;


h q0;
measure q0 -> meas0;
'''

In [3]:
with open("RyLength2.json", "r") as file:
    jobs = json.load(file)

In [4]:
def checks(r,i):
    if (r['init0'][i] != '00'):
        return False
    if (r['hcheck0'][i] != '00'):
        return False
    meas0 = r['meas0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False


    if (r['hcheck1'][i] != '00'):
       return False
    meas0 = r['magic0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False

    meas0 = r['magic1'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False
    
    if (r['ycheck0'][i] != '0'):
        return False
    if (r['xcheck0'][i] != '00'):
        return False
    if (r['flags0'][i] != '00'):
        return False
    if (r['ycheck1'][i] != '0'):
        return False
    if (r['xcheck1'][i] != '00'):
       return False
    if (r['flags1'][i] != '00'):
       return False
    return True


def non_gate_checks(r,i):
    if (r['init0'][i] != '00'):
        return False
    if (r['hcheck0'][i] != '00'):
        return False
    meas0 = r['meas0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False
    if (r['hcheck1'][i] != '00'):
       return False
    if (r['ycheck0'][i] != '0'):
        return False   
    if (r['ycheck1'][i] != '0'):
        return False
    return True

def success(r,i):
    meas0 = r['meas0'][i]
    return (int(meas0[5]) + int(meas0[3]) + int(meas0[1]))%2 == 1

In [5]:
totals = 0
totals_no_gate_checks = 0
failures = 0
shots = 0
for j in jobs:
    r = j['results']
    for i in range(0,len(r['hcheck1'])):
        shots += 1
        if (checks(r,i)):
            totals += 1
            if (not success(r,i)):
                failures += 1
        if (non_gate_checks(r,i)):
            totals_no_gate_checks += 1
print('failures:', failures, 'totals:', totals)
print('logical failure rate:', failures/totals)
print('total accept rate:', totals/shots, 'standard deviation:', math.sqrt((totals/shots)*(shots-totals))/shots)
print('accept rate of gate teleportations:', totals/totals_no_gate_checks, 'standard deviation:', math.sqrt((totals/totals_no_gate_checks)*(totals_no_gate_checks-totals))/totals_no_gate_checks)

failures: 1 totals: 7577
logical failure rate: 0.00013197835554968986
total accept rate: 0.7577 standard deviation: 0.00428474865073787
accept rate of gate teleportations: 0.9040687268822336 standard deviation: 0.003216865529418345


In [6]:
p = failures/totals
n = totals
wilson = (1/(1 + 1/n))*(1/(2*n))*(math.sqrt(4*n*p*(1-p) + 1))
wilson_mean = 1/(1+ (1/n))*(p + (1/(2*n)))
print('logical failure rate confidence interval: (',  wilson_mean - wilson, ',',  wilson_mean + wilson, ')',)

logical failure rate confidence interval: ( 5.041238259739241e-05 , 0.00034547043608827656 )
